In [37]:
import numpy as np
import os
from typing import List, Tuple, Dict
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
font = {'size': 16}

matplotlib.rc('font', **font)

In [38]:
DATASET_PATH = "../dataset"
RESULT_ROOT_PATH = "results"
MODEL_NAME = "lstm"
# MODEL_NAME = "nlinear"
# MODEL_NAME = "segrnn"
# MODEL_NAME = "patchtst"

OFFLINE_TRAINING_PATH="offline_training"
ONLINE_TRAINING_PATH="online_training"

IMG_SAVE_ROOT="img_comparison"

In [39]:
def get_data_file_list(dataset_path: str) -> List[str]:
    return os.listdir(dataset_path)

In [40]:
def read_offline_result(file_name: str) -> np.ndarray:
    dir = os.path.join(OFFLINE_TRAINING_PATH, os.path.join(RESULT_ROOT_PATH, MODEL_NAME))
    result = None
    with open(os.path.join(dir, file_name), "r") as f:
        result = json.load(f)
    result = np.array(result)
    return result

In [41]:
def read_online_result(file_name: str) -> np.ndarray:
    dir = os.path.join(ONLINE_TRAINING_PATH, os.path.join(RESULT_ROOT_PATH, MODEL_NAME))
    result = None
    with open(os.path.join(dir, file_name), "r") as f:
        result = json.load(f)
    result = np.array(result)
    return result

In [42]:
def get_offline_gt_and_pd(workload_name: str, name_split: str, is_scaled: bool) -> Tuple[np.ndarray, np.ndarray]:
    gt_file_name = None
    if is_scaled:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"scaled.json"
    else:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"original.json"

    pd_file_name = None
    if is_scaled:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"scaled.json"
    else:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"original.json"

    gt = read_offline_result(gt_file_name)
    pd = read_offline_result(pd_file_name)

    gt_0 = gt[:, 0, :]
    pd_0 = pd[:, 0, :]
    return gt_0, pd_0

In [43]:
def get_online_gt_and_pd(workload_name: str, name_split: str, is_scaled: bool) -> Tuple[np.ndarray, np.ndarray]:
    gt_file_name = None
    if is_scaled:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"scaled.json"
    else:
        gt_file_name = workload_name+"_gt_"+name_split+"_"+"original.json"

    pd_file_name = None
    if is_scaled:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"scaled.json"
    else:
        pd_file_name = workload_name+"_pd_"+name_split+"_"+"original.json"

    gt = read_online_result(gt_file_name)
    pd = read_online_result(pd_file_name)
    
    gt_0=gt[:,:,0,:]
    pd_0=pd[:,:,0,:]
    return gt_0, pd_0

In [44]:
def plot_comparison(gt: np.ndarray, offline_pd: np.ndarray, online_pd: np.ndarray,title=None):
    color_gt, color_offline_pd, color_online_pd = "#3F51B5", "#9C27B0", "#FF9800" # material indigo, purple, orange
    fig, ax = plt.subplots()
    fig.set_size_inches(9,3)
    x = range(len(gt))
    ax.plot(x, gt, color=color_gt, label="gt")
    ax.plot(x, offline_pd, color=color_offline_pd, label="pd_offline_online")
    ax.plot(x, online_pd, color=color_online_pd, label="pd_sliding_window")
    ax.set_xlabel("time (min)")
    ax.set_ylabel("workload x (10 requests)")
    ax.grid(True,linestyle="--")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend()
    if title is not None:
        ax.set_title(title)
    return fig, ax

In [45]:
data_file_list = get_data_file_list(DATASET_PATH)
is_scaled = False
for file_name in data_file_list:
    workload_name = file_name.split(".")[0]
    offline_gt, offline_pd = get_offline_gt_and_pd(workload_name, "test", is_scaled)
    online_gt, online_pd = get_online_gt_and_pd(workload_name, "test", is_scaled)
    start_index = len(online_gt)-len(offline_gt)
    fig_name = "workload_test "+workload_name.split("_")[1]
    fig, ax = plot_comparison(online_gt[start_index:].squeeze()/10, offline_pd.squeeze()/10, online_pd[start_index:].squeeze()/10, title=fig_name)
    if not os.path.exists(os.path.join(IMG_SAVE_ROOT, MODEL_NAME)):
        os.makedirs(os.path.join(IMG_SAVE_ROOT, MODEL_NAME))
    fig.savefig(os.path.join(os.path.join(IMG_SAVE_ROOT, MODEL_NAME), workload_name+".pdf"))
    plt.close()